# Tutorial 4: Function Secret Sharing
In our library, we also support Function Secret Sharing, which refers to Boyle et al. 's paper: Function Secret Sharing: Improvements and Extensions.2016; Function Secret Sharing for Mixed-Mode and Fixed-Point Secure Computation.2021
Function secret sharing is mainly the secret sharing of three functions, distributed point function (DPF), distributed comparison function (DCF) and distributed interval containment function (DICF).
In addition, we also refer to Storrier et al. 's paper: Grotto: Screaming fast (2+ 1)-PC for ℤ2n via (2, 2)-DPFs to implement the functionality of DICF by DPF

## DPF
Definition：
$$
f_{\alpha,\beta}(x)=\begin{cases}
\beta, & \text {if x=$\alpha$} \\
0, & \text {else}
\end{cases}
$$
We need to ask for the value of the function at the x position, and at the same time we need to hide $\alpha$ and $\beta$. The scheme adopted is to have the DPF key over $\alpha$ and $\beta$ generated by a trusted third party, and then to distribute the key to the two parties, who each use the key to calculate the value of the function when the independent variable is x.

In [1]:
# import the libraries
from crypto.primitives.function_secret_sharing import DPF
from common.tensor import RingTensor

num_of_keys = 10  # We need a few keys for a few function values, but of course we can generate many keys in advance.

# generate keys in offline phase
# set alpha and beta
alpha = RingTensor.convert_to_ring(5)
beta = RingTensor.convert_to_ring(1)

Key0, Key1 = DPF.gen(num_of_keys=num_of_keys, alpha=alpha, beta=beta)

# online phase
# generate some values what we need to evaluate
x = RingTensor.convert_to_ring([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# Party 0:
res_0 = DPF.eval(x=x, keys=Key0, party_id=0)

# Party 1:
res_1 = DPF.eval(x=x, keys=Key1, party_id=1)

# restore result
res = res_0 + res_1
print(res)

RingTensor
 value:tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0], device='cuda:0') 
 dtype:int 
 scale:1


## DCF
Definition：
$$
f_{\alpha,\beta}(x)=\begin{cases}
\beta, & \text {if x < $\alpha$} \\
0, & \text {else}
\end{cases}
$$
The method for computing the DCF value is similar to that of DPF.

In [2]:
# import the libraries
from crypto.primitives.function_secret_sharing import DCF
from common.tensor import RingTensor

num_of_keys = 10  # We need a few keys for a few function values, but of course we can generate many keys in advance.

# generate keys in offline phase
# set alpha and beta
alpha = RingTensor.convert_to_ring(5)
beta = RingTensor.convert_to_ring(1)

Key0, Key1 = DCF.gen(num_of_keys=num_of_keys, alpha=alpha, beta=beta)

# online phase
# generate some values what we need to evaluate
x = RingTensor.convert_to_ring([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# Party 0:
res_0 = DCF.eval(x=x, keys=Key0, party_id=0)

# Party 1:
res_1 = DCF.eval(x=x, keys=Key1, party_id=1)

# restore result
res = res_0 + res_1
print(res)

RingTensor
 value:tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0], device='cuda:0') 
 dtype:int 
 scale:1


## DICF
definition：
$$
f_{p,q}(x)=\begin{cases}
1, & \text {if p$\leq$x $\leq$ q} \\
0, & \text {else}
\end{cases}
$$
We set the function value to 1 when x falls within the range of p and q. With DICF, we can achieve a comparison between two numbers. For two numbers, x and y, if the difference between them falls within the positive range, it indicates that x is greater than y.
Now we will demonstrate how to compute the value of the DICF for a given input x. However, unlike DPF and DCF, the DICF hides the information of the input x, rather than the information of the upper and lower bounds. This is done to serve the purpose of performing comparison.

In [3]:
# import the libraries
from crypto.protocols.function_secret_sharing import DICF
from common.tensor import RingTensor

# generate key in offline phase
num_of_keys = 10
down_bound = RingTensor(3)
upper_bound = RingTensor(7)

Key0, Key1 = DICF.gen(num_of_keys=num_of_keys, down_bound=down_bound, upper_bound=upper_bound)

# evaluate x in online phase
# generate some values what we need to evaluate
x = RingTensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
x_shift = x + Key0.r_in.reshape(x.shape) + Key1.r_in.reshape(x.shape)

# online phase
# Party 0:
res_0 = DICF.eval(x_shift=x_shift, keys=Key0, party_id=0, down_bound=down_bound, upper_bound=upper_bound)

# Party 1:
res_1 = DICF.eval(x_shift=x_shift, keys=Key1, party_id=1, down_bound=down_bound, upper_bound=upper_bound)

# restore result
res = res_0 + res_1
print(res)

RingTensor
 value:tensor([0, 0, 1, 1, 1, 1, 1, 0, 0, 0], device='cuda:0') 
 dtype:int 
 scale:1


### GROTTO
The paper GROTTO proposes a scheme to implement DICF faster with prefix sum and DPF. The following is the sample code. It should be noted that although the result evaluated by this scheme is 0,1, it actually corresponds to bool values False and True, that is to say, the result of this scheme is a Boolean secret share value, while the previous schemes are arithmetic secret share values.

In [4]:
# import the libraries
from crypto.protocols.function_secret_sharing import PPQCompare
from common.tensor import RingTensor

# generate key in offline phase
num_of_keys = 10
down_bound = RingTensor(3)
upper_bound = RingTensor(7)
beta = RingTensor(1)

Key0, Key1 = PPQCompare.gen(num_of_keys=num_of_keys, beta=beta)

# evaluate x in online phase
# generate some values what we need to evaluate
x = RingTensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
x_shift = Key0.r_in.reshape(x.shape) + Key1.r_in.reshape(x.shape) - x

# online phase
# Party 0:
res_0 = PPQCompare.eval(x_shift=x_shift, key=Key0, party_id=0, down_bound=down_bound, upper_bound=upper_bound)

# Party 1:
res_1 = PPQCompare.eval(x_shift=x_shift, key=Key1, party_id=1, down_bound=down_bound, upper_bound=upper_bound)

# restore result
res = res_0 ^ res_1
print(res)

RingTensor
 value:tensor([0, 0, 1, 1, 1, 1, 0, 0, 0, 0], device='cuda:0') 
 dtype:int 
 scale:1


## The application of FSS
As mentioned before, DICF can be used for number comparison. If you want to use DICF for number comparison, please change `GE_TYPE` in`./config/base_configs.py` to `DICF`, and then refer to the method in Tutorial_2 to achieve it. We also talked about an improved FSS method in Tutorial_0. If you want to use this method for number comparison, change `GE_TYPE` to `GROTTO` and then refer to Tutorial_2.